In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Reshape

In [19]:
# Load and preprocess your image dataset
train_dir = 'C:/Users/singt/Downloads/dataset_18'
validation_dir = 'C:/Users/singt/Downloads/dataset_18'

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [20]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 480 images belonging to 4 classes.
Found 480 images belonging to 4 classes.


In [21]:
# Define the ResNet model
def resnet_model():
    inputs = keras.Input(shape=(224, 224, 3))

    x = layers.Conv2D(64, 7, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)  # Output shape: (112, 112, 64)

    x = resnet_block(x, 64, 2)  # Output shape: (56, 56, 64)
    x = resnet_block(x, 128, 2)  # Output shape: (28, 28, 128)
    x = resnet_block(x, 256, 2)  # Output shape: (14, 14, 256)
    x = resnet_block(x, 512, 2)  # Output shape: (7, 7, 512)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(10, activation='softmax')(x)

    model = keras.Model(inputs, x)
    return model

def resnet_block(x, filters, stride):
    y = layers.Conv2D(filters, 3, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv2D(filters, 3, padding='same')(y)
    y = layers.BatchNormalization()(y)

    if stride != 1:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(x)
    else:
        shortcut = x

    shortcut = layers.BatchNormalization()(shortcut)

    y = layers.Add()([y, shortcut])
    y = layers.Activation('relu')(y)

    return y

In [22]:
# Compile the model
model = resnet_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "c:\Users\singt\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\singt\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\singt\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\singt\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\singt\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\singt\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py"

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy:.2f}')

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Define the ResNet block
def resnet_block(x, filters, stride):
    y = layers.Conv2D(filters, 3, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv2D(filters, 3, padding='same')(y)
    y = layers.BatchNormalization()(y)

    if stride != 1:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(x)
    else:
        shortcut = x

    shortcut = layers.BatchNormalization()(shortcut)

    y = layers.Add()([y, shortcut])
    y = layers.Activation('relu')(y)

    return y

# Define the ResNet model
def resnet_model(num_classes):
    inputs = keras.Input(shape=(224, 224, 3))

    x = layers.Conv2D(64, 7, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)

    x = resnet_block(x, 64, 2)
    x = resnet_block(x, 128, 2)
    x = resnet_block(x, 256, 2)
    x = resnet_block(x, 512, 2)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, x)
    return model

# Define the data generators
train_dir = 'C:/Users/singt/Downloads/dataset_18'
validation_dir = 'C:/Users/singt/Downloads/dataset_18'

train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Compile the model
model = resnet_model(4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, steps_per_epoch=train_generator.samples // 32, epochs=10, validation_data=validation_generator, validation_steps=validation_generator.samples // 32)

Found 480 images belonging to 4 classes.
Found 480 images belonging to 4 classes.
Epoch 1/10
15/15 [==============================] - 30s 2s/step - loss: 1.2984 - accuracy: 0.5813 - val_loss: 7.0203 - val_accuracy: 0.2500
Epoch 2/10
15/15 [==============================] - 28s 2s/step - loss: 0.7377 - accuracy: 0.7375 - val_loss: 1.4388 - val_accuracy: 0.3167
Epoch 3/10
15/15 [==============================] - 27s 2s/step - loss: 0.6260 - accuracy: 0.7708 - val_loss: 1.6956 - val_accuracy: 0.2333
Epoch 4/10
15/15 [==============================] - 27s 2s/step - loss: 0.6191 - accuracy: 0.7646 - val_loss: 2.0485 - val_accuracy: 0.3979
Epoch 5/10
15/15 [==============================] - 27s 2s/step - loss: 0.4457 - accuracy: 0.8313 - val_loss: 2.4025 - val_accuracy: 0.2750
Epoch 6/10
15/15 [==============================] - 27s 2s/step - loss: 0.3789 - accuracy: 0.8521 - val_loss: 2.8363 - val_accuracy: 0.3917
Epoch 7/10
15/15 [==============================] - 28s 2s/step - loss: 0.3248